In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action='ignore') #ignore all warnings include SSL warning for reqest.get

In [32]:
headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

In [33]:
#print error messages of every dead link under a given page
def printError(url,websiteName):
    deadLinksOut = open(websiteName+"deadLinks.txt", "a")
    timeOutLinksOut = open(websiteName+"timeOutLinks.txt", "a")
    errorFound = False
    try:
        req = requests.get(url, headers=headers, timeout = 20)
        req.raise_for_status()
    except Exception:
        return
    soup = BeautifulSoup(req.content, 'html.parser')
    content = soup.find_all("div", {"class": "region-content"})[0]
    for link in content.find_all('a'):
        linkUrl = link.get('href')
        text = link.get_text()
        if linkUrl == None:
            continue
        if "http" not in linkUrl:
            if "mailto:" or "tel:" in linkUrl:
                continue
            linkUrl = website + linkUrl
        try:
            r = requests.get(linkUrl, headers=headers, verify=False, timeout = 20)
            r.raise_for_status()
        except Exception as e:
            if e == requests.exceptions.Timeout:
                timeOutLinksOut.write('\t'+text+'\n')
                timeOutLinksOut.write('\t'+linkUrl+'\n')
                timeOutLinksOut.write("\n")
            if not errorFound:
                deadLinksOut.write(url)
                errorFound = True
            deadLinksOut.write('\t'+text+'\n')
            deadLinksOut.write('\t'+linkUrl+'\n')
            deadLinksOut.write('\n')
            # print('\t'+str(e))
    if errorFound:
       deadLinksOut.write('\n')
    deadLinksOut.close()
    timeOutLinksOut.close()

In [34]:
website = "https://studentlife.umich.edu/parents"
req = requests.get(website+"/sitemap.xml", headers=headers, timeout = 20)
soup = BeautifulSoup(req.content, 'html.parser')
websiteName = website[website.find("//")+2:].replace("/","_")

In [35]:
urls = []
for link in soup.find_all('loc'):
    urls.append(link.string)


In [36]:
for i in tqdm_notebook(range(len(urls))):
    printError(urls[i],websiteName)

  0%|          | 0/428 [00:00<?, ?it/s]

KeyboardInterrupt: 